In [1]:
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
iOption0 = 0

In [3]:
N_centr = 5
eIDbins = [3,4,5]
primaryID = [[18,19,20,23,24,25],[19,20,24,25],[20,25]]
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,20,20,40,40,60,60,80,80,93]
real_file_path = 'input/'
real_file_names=['my-m_ee_Run14AuAu_39th_new_19579_900runs.root']
sim_file_path="../sim/output/Newembed/sngl/Old/"
primary_file_names=['ccbar_50M.root']
hadron_file_names=['piminus_100M.root']
dalitz_file_names=['dalitz_50M.root']
conv_file_names=['conv_100M.root']
sim_file_names = [primary_file_names,hadron_file_names,dalitz_file_names,conv_file_names ]
part_names = ["c#bar{c}","dalitz","#gamma #rightarrow ee","#pi^{-}"]
hist_names = ["el_pt_hist_"]

In [4]:
hists_read_real, hists_read_sim = [], []

for iFilereal in range(len(real_file_names)):
    infile = root.TFile.Open(real_file_path+real_file_names[iFilereal], "read")
    hists_read_real_ifile = []
    for icentr in range(N_centr):
        hists_read_real_ifile.append(infile.Get(hist_names[0]+f'{icentr}'))
        hists_read_real_ifile[-1].SetDirectory(root.nullptr)
        hists_read_real_ifile[-1].SetName(hist_names[0]+f'{icentr}'+'_real')
    hists_read_real.append(hists_read_real_ifile)
    infile.Close()

for iFilesim in range(len(sim_file_names)):
    infile = root.TFile.Open(sim_file_path+sim_file_names[iFilesim][0], "read")
    hists_read_sim_ifile = []
    for icentr in range(N_centr):
        hists_read_sim_ifile.append(infile.Get(hist_names[0]+f'{icentr}'))
        hists_read_sim_ifile[-1].SetDirectory(root.nullptr)
        hists_read_sim_ifile[-1].SetName(hist_names[0]+f'{icentr}'+f'_sim{iFilesim}_')
    hists_read_sim.append(hists_read_sim_ifile)
    infile.Close()

In [5]:
hists_pt_real, hists_pt_sim = [], []

for iFilereal in range(len(real_file_names)):
    hists_pt_real_eid = []
    for ieID in range(len(eIDbins)):
        hists_pt_real_conv = []
        for iconv in range(len(primaryID)):
            hists_pt_real_centr = []
            for icentr in range(N_centr):
                hists_pt_real_centr.append(hists_read_real[0][icentr].ProjectionX(hists_read_real[iFilereal][icentr].GetName()+f"_pr{ieID}_{iconv}_0",eIDbins[ieID],eIDbins[ieID],primaryID[iconv][0],primaryID[iconv][0]))
                for jconv in range(1,len(primaryID[iconv])):
                    hists_pt_real_centr[-1].Add(hists_read_real[0][icentr].ProjectionX(hists_read_real[iFilereal][icentr].GetName()+f"_pr{ieID}_{iconv}_{jconv}",eIDbins[ieID],eIDbins[ieID],primaryID[iconv][jconv],primaryID[iconv][jconv]))
            hists_pt_real_conv.append(hists_pt_real_centr)
        hists_pt_real_eid.append(hists_pt_real_conv)
    hists_pt_real.append(hists_pt_real_eid)
    

for iFilesim in range(len(sim_file_names)):
    hists_pt_sim_eid = []
    for ieID in range(len(eIDbins)):
        hists_pt_sim_conv = []
        for iconv in range(len(primaryID)):
            hists_pt_sim_centr = []
            for icentr in range(N_centr):
                hists_pt_sim_centr.append(hists_read_sim[iFilesim][icentr].ProjectionX(hists_read_sim[iFilesim][icentr].GetName()+f"_pr{ieID}_{iconv}_0",eIDbins[ieID],eIDbins[ieID],primaryID[iconv][0],primaryID[iconv][0]))
                for jconv in range(1,len(primaryID[iconv])):
                    hists_pt_sim_centr[-1].Add(hists_read_sim[iFilesim][icentr].ProjectionX(hists_read_sim[iFilesim][icentr].GetName()+f"_pr{ieID}_{iconv}_{jconv}",eIDbins[ieID],eIDbins[ieID],primaryID[iconv][jconv],primaryID[iconv][jconv]))
            hists_pt_sim_conv.append(hists_pt_sim_centr)
        hists_pt_sim_eid.append(hists_pt_sim_conv)
    hists_pt_sim.append(hists_pt_sim_eid)

In [6]:
if False:
    c0 = root.TCanvas(f"c0",f"c0",1400,1400)
    c0.Divide(2,3)
    for icentr in range(N_centr):
        c0.cd(icentr+1)
        root.gPad.SetLogy()
        hists_pt_sim[1][0][0][icentr].Draw()
    c0.Draw()

signal = electrons + hadron => 1 = (1-x) + x

applying cut: signal, electrons and hadraons are reduced diffrently

signal_fraction = electron_fraction * (1-x) + hadron_fraction * x

x = ( electron_fraction - signal_fraction ) / ( electron_fraction - hadron_fraction )

In [7]:
rebin = 1
for icentr in range(N_centr):
    for ieID in range(0,len(eIDbins)):
        for iconv in range(len(primaryID)):
            hists_pt_real[0][ieID][iconv][icentr].Rebin(rebin)
            hists_pt_sim[0][ieID][iconv][icentr].Rebin(rebin)
            hists_pt_sim[1][ieID][iconv][icentr].Rebin(rebin)

In [8]:
graphs= [] 
for icentr in range(N_centr):
    graphs_centr = []
    for ieID in range(1,len(eIDbins)):
        graphs_eid = []
        for iconv in range(len(primaryID)):
            graphs_conv = []
            for iptbin in range(1,1+hists_pt_real[0][ieID][iconv][icentr].GetNbinsX()):
                raw_signal_frac =  hists_pt_real[0][ieID][iconv][icentr].GetBinContent(iptbin)/(hists_pt_real[0][0][iconv][icentr].GetBinContent(iptbin)+1e-6)
                raw_primary_frac = hists_pt_sim[0][ieID][iconv][icentr].GetBinContent(iptbin)/(hists_pt_sim[0][0][iconv][icentr].GetBinContent(iptbin)+1e-6)
                raw_hadron_frac =  hists_pt_sim[1][ieID][iconv][icentr].GetBinContent(iptbin)/(hists_pt_sim[1][0][iconv][icentr].GetBinContent(iptbin)+1e-6)
                #print(raw_signal_frac, raw_primary_frac, raw_hadron_frac)
                if (raw_primary_frac - raw_signal_frac) == 0: hadron_frac = 0
                else: 
                    if raw_primary_frac - raw_hadron_frac > 0:
                        hadron_frac = ( raw_primary_frac - raw_signal_frac ) / ( raw_primary_frac - raw_hadron_frac )
                    else: hadron_frac = -1
                    if hadron_frac<0: hadron_frac = 0
                    if hadron_frac>1: hadron_frac = 1
                hadron_frac_err = hadron_frac * (hists_pt_sim[1][ieID][iconv][icentr].GetBinError(iptbin)/(hists_pt_sim[1][ieID][iconv][icentr].GetBinContent(iptbin)+1e-6))*2**0.5
                    #print(hadron_frac)
                graphs_conv.append([hists_pt_real[0][ieID][iconv][icentr].GetBinCenter(iptbin),hadron_frac,hadron_frac_err])
            graphs_eid.append(graphs_conv)
        graphs_centr.append(graphs_eid)
    graphs.append(graphs_centr)
print(graphs)

[[[[[0.05, 0, 0.0], [0.15000000000000002, 1.0, 0.0], [0.25, 1, 1.414212148160947], [0.35000000000000003, 1, 0.5773501729645969], [0.45, 0.06279502090087098, 0.010186697396439737], [0.55, 0.06769801737074244, 0.010703996264659532], [0.6500000000000001, 0.0694772135743858, 0.012003834041695211], [0.7500000000000001, 0.10024192496528274, 0.019115384718119726], [0.8500000000000001, 0.0522365892522038, 0.010553384494993275], [0.9500000000000001, 0.058396493462918954, 0.013576908585022938], [1.05, 0.050196132682033046, 0.012357433251988048], [1.1500000000000001, 0.11895588429325497, 0.027290350775191596], [1.2500000000000002, 0.11356321708695183, 0.03348796429308838], [1.35, 0.2561686546786492, 0.07245543424206577], [1.4500000000000002, 0.41207437939502944, 0.11655223054729616], [1.55, 1, 0.3244428251855869], [1.6500000000000001, 0, 0.0], [1.7500000000000002, 0.22062654973529724, 0.08653684582923722], [1.85, 1, 0.47140446841275746], [1.9500000000000002, 0.20055937910875565, 0.094544587493955

In [9]:
Tgraphs= [] 
for icentr in range(N_centr):
    graphs_centr = []
    for ieID in range(0,len(eIDbins)-1):
        graphs_eid = []
        for iconv in range(len(primaryID)):
            graphs_conv = root.TGraphErrors()
            ipoint = 0
            for iptbin in range(0,0+hists_pt_real[0][ieID][iconv][icentr].GetNbinsX()):
                if hists_pt_real[0][ieID][iconv][icentr].GetBinCenter(iptbin+1)<0.4: continue
                if hists_pt_real[0][ieID][iconv][icentr].GetBinCenter(iptbin+1)>2.0: continue
                graphs_conv.AddPoint(hists_pt_real[0][ieID][iconv][icentr].GetBinCenter(iptbin+1),graphs[icentr][ieID][iconv][iptbin][1])
                graphs_conv.SetPointError(ipoint,hists_pt_real[0][ieID][iconv][icentr].GetBinWidth(iptbin+1)/2,graphs[icentr][ieID][iconv][iptbin][2])
                ipoint+=1

            graphs_eid.append(graphs_conv)
        graphs_centr.append(graphs_eid)
    Tgraphs.append(graphs_centr)

In [10]:
c0 = root.TCanvas(f"c0",f"c0",1400,1400*2)
c0.Divide(3,6)
h = []
for icentr in range(N_centr):
    graphs_centr = []
    for ieID in range(0,len(eIDbins)-1):
        graphs_eid = []
        for iconv in range(len(primaryID)):
            c0.cd(icentr+1+6*iconv)
            if ieID == 0 :
                h.append(root.TH1D(f"name{icentr}{ieID}{iconv}", f"name{icentr}{ieID}{iconv}", 100, 0.1, 1.9))
                h[-1].SetMaximum(0.19)
                Format_Hist_total(h[-1], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
                                      Mstyle=21,  Msize=0, Mcolor=1,  Lwidth=6,  Lcolor=1,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
                h[-1].Draw()
                Format_Graph( Tgraphs[icentr][ieID][iconv],  Mstyle=21,  Msize=1,  Mcolor=1,  Lwidth=1,  Lcolor=1,  Malpha=1,  Lalpha=1)
                Tgraphs[icentr][ieID][iconv].Draw("same P")
            else :
                Format_Graph( Tgraphs[icentr][ieID][iconv],  Mstyle=20,  Msize=1,  Mcolor=2,  Lwidth=1,  Lcolor=1,  Malpha=2,  Lalpha=1)
                Tgraphs[icentr][ieID][iconv].Draw("same P")
c0.Draw()

In [11]:
hists_pt_real, hists_pt_sim = [], []

for iFilereal in range(len(real_file_names)):
    hists_pt_real_eid = []
    for ieID in range(len(eIDbins)):
        hists_pt_real_conv = []
        for iconv in range(len(primaryID)):
            hists_pt_real_centr = []
            for icentr in range(N_centr):
                hists_pt_real_centr.append(hists_read_real[0][icentr].ProjectionX(hists_read_real[iFilereal][icentr].GetName()+f"_p1r{ieID}_{iconv}_0",eIDbins[ieID],eIDbins[ieID],primaryID[iconv][0],primaryID[iconv][0]))
                for jconv in range(1,len(primaryID[iconv])):
                    hists_pt_real_centr[-1].Add(hists_read_real[0][icentr].ProjectionX(hists_read_real[iFilereal][icentr].GetName()+f"_p1r{ieID}_{iconv}_{jconv}",eIDbins[ieID],eIDbins[ieID],primaryID[iconv][jconv],primaryID[iconv][jconv]))
            hists_pt_real_conv.append(hists_pt_real_centr)
        hists_pt_real_eid.append(hists_pt_real_conv)
    hists_pt_real.append(hists_pt_real_eid)
    

for iFilesim in range(len(sim_file_names)):
    hists_pt_sim_eid = []
    for ieID in range(len(eIDbins)):
        hists_pt_sim_conv = []
        for iconv in range(len(primaryID)):
            hists_pt_sim_centr = []
            for icentr in range(N_centr):
                hists_pt_sim_centr.append(hists_read_sim[iFilesim][icentr].ProjectionX(hists_read_sim[iFilesim][icentr].GetName()+f"_p1r{ieID}_{iconv}_0",eIDbins[ieID],eIDbins[ieID],primaryID[iconv][0],primaryID[iconv][0]))
                for jconv in range(1,len(primaryID[iconv])):
                    hists_pt_sim_centr[-1].Add(hists_read_sim[iFilesim][icentr].ProjectionX(hists_read_sim[iFilesim][icentr].GetName()+f"_p1r{ieID}_{iconv}_{jconv}",eIDbins[ieID],eIDbins[ieID],primaryID[iconv][jconv],primaryID[iconv][jconv]))
            hists_pt_sim_conv.append(hists_pt_sim_centr)
        hists_pt_sim_eid.append(hists_pt_sim_conv)
    hists_pt_sim.append(hists_pt_sim_eid)

In [12]:
graphs= [] 
for icentr in range(N_centr):
    graphs_centr = []
    for ieID in range(1,len(eIDbins)):
        graphs_eid = []
        for iconv in range(len(primaryID)):
            raw_signal_frac =  hists_pt_real[0][ieID][iconv][icentr].Integral(5,10)/(hists_pt_real[0][0][iconv][icentr].Integral(5,10)+1e-6)
            raw_primary_frac = hists_pt_sim[0][ieID][iconv][icentr].Integral(5,10)/(hists_pt_sim[0][0][iconv][icentr].Integral(5,10)+1e-6)
            raw_hadron_frac =  hists_pt_sim[1][ieID][iconv][icentr].Integral(5,10)/(hists_pt_sim[1][0][iconv][icentr].Integral(5,10)+1e-6)
            #print(raw_signal_frac, raw_primary_frac, raw_hadron_frac)
            if (raw_signal_frac - raw_primary_frac) == 0: hadron_frac = 0
            else: 
                if raw_primary_frac - raw_hadron_frac > 0:
                    hadron_frac = ( raw_primary_frac - raw_signal_frac ) / ( raw_primary_frac - raw_hadron_frac )
                else: hadron_frac = -1
                if hadron_frac<0: hadron_frac = 0
                if hadron_frac>1: hadron_frac = 1
            graphs_eid.append(hadron_frac)
        graphs_centr.append(graphs_eid)
    graphs.append(graphs_centr)
print(graphs)

[[[0.15603991738842993, 0.16038703359856762, 0.16660620487034358], [0.22094803064740143, 0.23453549900987197, 0.23886303838766137]], [[0.10928507687870619, 0.11426513166527738, 0.11442227328443161], [0.14234618459819584, 0.1432008882681815, 0.14853976406299121]], [[0.055153181125772235, 0.059366917483318335, 0.06386357395078694], [0.0702236138284098, 0.07427299609907387, 0.07876526615150209]], [[0, 0.0018343664854549872, 0.0047634846770248825], [0.012891220473722676, 0.014787168197471574, 0.01841531042907168]], [[0, 0, 0], [0, 0, 0]]]


In [13]:
if True :
    c0 = root.TCanvas(f"c0",f"c0",1400,1400*2)
    c0.Divide(3,6)
    for icentr in range(N_centr):
        for iconv in range(len(primaryID)):
            c0.cd(icentr+1+6*iconv)
            root.gPad.SetLogy()
                
            Format_Hist_total(hists_pt_real[0][0][iconv][icentr], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
                      Mstyle=21,  Msize=2, Mcolor=2,  Lwidth=4,  Lcolor=2,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
            hists_pt_real[0][0][iconv][icentr].Draw()
    
            for iptbin in range(1,1+hists_pt_real[0][0][iconv][icentr].GetNbinsX()):
                hists_pt_sim[1][0][iconv][icentr].SetBinContent(iptbin,graphs[icentr][0][iconv]*hists_pt_real[0][0][iconv][icentr].GetBinContent(iptbin))
                hists_pt_sim[1][1][iconv][icentr].SetBinContent(iptbin,graphs[icentr][1][iconv]*hists_pt_real[0][0][iconv][icentr].GetBinContent(iptbin))
                
            Format_Hist_total(hists_pt_sim[1][0][iconv][icentr], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
                      Mstyle=21,  Msize=0, Mcolor=1,  Lwidth=6,  Lcolor=1,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
            Format_Hist_total(hists_pt_sim[1][1][iconv][icentr], title_x="p_{T} [GeV]",  title_y="dN/dp_{T}", left=0.15, bottom=0.15, right=0.01, top=0.01,  Tsize=0.07,  Lsize=0.06,\
                      Mstyle=21,  Msize=0, Mcolor=4,  Lwidth=3,  Lcolor=4,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
            hists_pt_sim[1][0][iconv][icentr].Draw('same H')
            hists_pt_sim[1][1][iconv][icentr].Draw('same H')
    c0.Draw()            